In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import lxml.etree as ET
from xml.dom import minidom
import pathlib
import glob
import datetime
import sys


In [2]:
def read_oaixml(xmlfile):
    doc = minidom.parse(xmlfile)
    authors=[]
    for element in doc.getElementsByTagName("dc:creator"):
        authors.append(element.firstChild.data)
  
    keywords=[]
    for element in doc.getElementsByTagName("dc:subject"):
        keywords.append(element.firstChild.data)
    
    try:
        category=doc.getElementsByTagName("dc:type")[1].firstChild.data
    except :
        category="None"
    
    try:
        lic=doc.getElementsByTagName("dc:rights")[1].firstChild.data
    except:
        lic="None"
    
    try:
        abstract=doc.getElementsByTagName("dc:description")[0].firstChild.data
    except:
        abstract="None"

    output={
        'category':category,
        'date':doc.getElementsByTagName("dc:date")[0].firstChild.data,
        'title':doc.getElementsByTagName("dc:title")[0].firstChild.data,
        'authors':authors,
        'abstract':abstract,
        'url':doc.getElementsByTagName("dc:identifier")[0].firstChild.data,
        'keywords':keywords,
        'license':lic,
    }
    return output

In [13]:
def makeprevurl(recordurl,filename):
    return recordurl+"/preview/"+filename

def getfiles(prevurl):
    files = pd.DataFrame(columns=['filename','extension','size'])

    try :
        r = requests.get(prevurl.replace('https','http'))
        prev=BeautifulSoup(r.text, 'html.parser')
        for t in prev.select(".fa-file-o"):
            k=t.find_previous("li").span
            files=files.append({'filename':k.contents[1].encode(encoding='UTF-8',errors='ignore'),
                          'extension':pathlib.Path(k.contents[1]).suffix.encode(encoding='UTF-8',errors='ignore'),
                          'size':k.next_sibling.next_sibling.contents[0].encode(encoding='UTF-8',errors='ignore')}
                         ,ignore_index=True)
    except :
        r=""
    return files

def getarchives(recordurl):

    files = pd.DataFrame(columns=['filename','extension','size','contents','nfile_archive'])
    try :
        r = requests.get(recordurl.replace('https','http'))
        page=BeautifulSoup(r.text, 'html.parser')
        for t in page.select(".filename"):
            k=t.find_previous("td")
            filename=k.a.contents[0]
            ext=pathlib.Path(filename).suffix
            if ext==".zip":
                archive=getfiles(makeprevurl(recordurl,filename))
            else :
                archive=pd.DataFrame(columns=['filename','extension','size'])

            files=files.append({
                'filename':filename.encode(encoding='UTF-8',errors='ignore'),
                'extension':ext.encode(encoding='UTF-8',errors='ignore'),
                'size':k.next_sibling.next_sibling.contents[0].encode(encoding='UTF-8',errors='ignore'),
                'contents':archive,
                'nfile_archive':archive.shape[0]
                }
                ,ignore_index=True)
    except :
        r=""
    return files


In [14]:
def time_elapsed(start):
    end = datetime.datetime.now()

    time_to_run = end - start
    minutes = int(time_to_run.seconds/60)
    seconds = time_to_run.seconds % 60
    return "Total runtime: " + str(minutes) + " minutes, " + str(seconds) + " seconds"

datadir='./xml-oai/'
fxml = glob.glob(datadir + "*.xml")
i=1
zenododata=pd.DataFrame(columns=['category','date','title','authors', 'abstract','keywords','license','url','files'])
date=datetime.datetime.now().strftime('%Y%m%d')
pathlib.Path("processed_data/" + date).mkdir(parents=True, exist_ok=True) 


In [ ]:
#file = open("status.dat", "r") 
#n=int(file.read())
#print(n)
#i=n
#zenododata=pd.read_pickle("processed_data/" + date + "/zenododata.pkl")


In [15]:
start=datetime.datetime.now()



fxml=fxml[i-1:]
print(len(fxml))
for xmlfile in fxml:
    row = read_oaixml(xmlfile)
    row.update({'files':getarchives(row['url'])})
    zenododata=zenododata.append(row,ignore_index=True)
    i += 1
    if zenododata.shape[0] % 1000 is 0:
        print(zenododata.shape[0], time_elapsed(start))
        f = open("status.dat", "w")
        f.write(str(i))
        f.close()
        zenododata.to_pickle("processed_data/" + date + "/zenododata.pkl")

5


In [ ]:
zenododata.to_pickle("processed_data/" + date + "/zenododata.pkl")

In [16]:

#zenododata
    
    

,category,date,title,authors,abstract,keywords,license,url,files
0,image-figure,2019-02-14,"Figures 20-34 from: Haddad CR, Henrard A, Jocq...","[Haddad, Charles R., Henrard, Arnaud, Jocqué, ...",Figures 20-34 Scanning electron microscope pho...,"[Animalia, Arthropoda, Arachnida, Araneae, Zod...",http://creativecommons.org/licenses/by/4.0/leg...,https://zenodo.org/record/2564337,filename extension size \...
1,image-photo,2019-05-09,Amaranthus hybridus L. (BR0000010604434),[Meise Botanic Garden],Belgium Herbarium image of Meise Botanic Garden.,"[Biodiversity, Taxonomy, Terrestrial, Herbariu...",http://creativecommons.org/licenses/by-sa/4.0/...,https://zenodo.org/record/2695517,filename extension size...
2,image-photo,2019-05-09,Arabis alpina L. subsp. caucasica (Willd.) Bri...,[Meise Botanic Garden],Belgium Herbarium image of Meise Botanic Garden.,"[Biodiversity, Taxonomy, Terrestrial, Herbariu...",http://creativecommons.org/licenses/by-sa/4.0/...,https://zenodo.org/record/2682111,filename extension size...
3,publication-article,2012-07-27,The Study of Synbiotic Dairy Products Rheologi...,"[Ilze Beitane, Inga Ciprovica]",The influence of lactulose and inulin on rheol...,"[Apparent viscosity, B.lactis, consistency coe...",https://creativecommons.org/licenses/by/4.0/,https://zenodo.org/record/1082963,filename extension size \ 0 b'...
4,publication-article,2017-12-31,Two new giant pill-millipede species of the ge...,"[Sagorny, Christina, Wesener, Thomas]","Sagorny, Christina, Wesener, Thomas (2017): Tw...","[Biodiversity, Taxonomy, Animalia, Arthropoda,...",None,https://zenodo.org/record/573095,"Empty DataFrame Columns: [filename, extension,..."


In [17]:
fxml = glob.glob(datadir + "*.xml")
#1.11*len(fxml)/(60*60*24)


In [ ]:
#1000/(60*15)

In [18]:
len(fxml)

245164